In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import torch 
from spanemo.model import SpanEmo
from spanemo.data_loader import DataClass, twitter_preprocessor
from fastprogress.fastprogress import format_time, master_bar, progress_bar
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import datetime
import numpy as np
import json
import math
import random

In [2]:
seed = 12345678
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

if str(device) == 'cuda:0':
    print("Currently using GPU: {}".format(device))
else:
    print("WARNING: USING CPU")

Currently using GPU: cuda:0


In [4]:
paths_dict = {
    "cornell": "../cornell_movie/predictions.csv",
    "iemocap": "../iemocap/predictions.csv", #do ur stuff here
    "bolt": "../bolt/predictions.csv", # do ur stuff here
}

df_dict = {
    "cornell": pd.read_csv(paths_dict["cornell"]),
    "iemocap": pd.read_csv(paths_dict["iemocap"]),
    "bolt": pd.read_csv(paths_dict["bolt"]),
}

text_dict = {
    "cornell": np.array(df_dict["cornell"]["utterance text"]),
    "iemocap": np.array(df_dict["iemocap"]["text"]), # whatever
    "bolt": np.array(df_dict["bolt"]["text"]),
}

dataset_name = "bolt"
data_path = paths_dict[dataset_name]
pred_df = df_dict[dataset_name]
text_arr = text_dict[dataset_name]

In [5]:
hyperparams = {
    'backbone':'bert-large-uncased',
    'max_length': 128, #maximum context length for the model
    'batch_size': 64,
}

In [9]:
model_path = "../../model1/models/final_model.pt"
model = SpanEmo(backbone=hyperparams["backbone"])
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
print()

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
x = 32768
i = 0
dataset = DataClass(hyperparams, text_arr[i*x:i*x+x], pred_mode=True)
data_loader = DataLoader(dataset,
                        batch_size=hyperparams['batch_size'],
                        shuffle=False
                        )

Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))
/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter_2018 - 1grams ...


PreProcessing dataset ...: 100%|██████████| 10237/10237 [00:09<00:00, 1040.76it/s]


In [10]:
index = 0

In [11]:
with torch.no_grad():
    current_index = 0
    for step, batch in enumerate(progress_bar(data_loader, parent=None, leave=False)):
        if step < index:
            current_index += hyperparams['batch_size']
            continue

        num_rows, y_pred, logits = model.predict(batch, device)
        
        pred_df.iloc[i*x + current_index: i*x + current_index + num_rows, pred_df.columns.get_loc("0"):] = logits.cpu().numpy()
        if step % 10 == 0 and step > 0: pred_df.to_csv(data_path, index=False)
        
        current_index += num_rows

pred_df.to_csv(data_path, index=False)

In [6]:
def loop(start=0, x=32768):
    model_path = "../../model1/models/final_model.pt"
    model = SpanEmo(backbone=hyperparams["backbone"])
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()

    preprocessor = twitter_preprocessor()
    
    mb = master_bar(range(start, math.ceil(len(pred_df)/x)))
    for i in mb:
        dataset = DataClass(hyperparams, 
                            text_arr[i*x : min(i*x+x, len(pred_df))], 
                            pred_mode=True, 
                            preprocessor=preprocessor,
                            pbar=lambda x, **kw: progress_bar(x, parent=mb, leave=False))
        data_loader = DataLoader(dataset,
                                batch_size=hyperparams['batch_size'],
                                shuffle=False
                                )
        
        with torch.no_grad():
            current_index = 0
            for step, batch in enumerate(progress_bar(data_loader, parent=mb, leave=False)):
                num_rows, y_pred, logits = model.predict(batch, device)

                pred_df.iloc[i*x + current_index: i*x + current_index + num_rows, pred_df.columns.get_loc("0"):] = logits.cpu().numpy()
                if step % 16 == 0 and step > 0: pred_df.to_csv(data_path, index=False)
                
                current_index += num_rows

        pred_df.to_csv(data_path, index=False)

In [11]:
loop(start=6)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format(

Reading twitter_2018 - 1grams ...
Reading twitter_2018 - 2grams ...
Reading twitter_2018 - 1grams ...


/home/alexlu/miniconda3/envs/emo/lib/python3.9/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
